In [10]:
%%configure -f
{ "conf":{    
          "spark.jars.packages": "io.delta:delta-core_2.12:0.8.0",
          "spark.sql.extensions": "io.delta.sql.DeltaSparkSessionExtension",
          "spark.sql.warehouse.dir": "s3://aws-poc-serverless-analytics/delta_lake_demo/clean/delta_dw/"
    
         }
}


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1622277616838_0033,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
7,application_1622277616838_0033,pyspark,idle,Link,Link,✔


In [14]:
target_table = "merchants_cln"
stg_table = "merchants_cln_stg"
database = "delta_lake_demo"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
spark.sql(f"DROP TABLE IF EXISTS merchants_cln ")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [9]:
spark.sql(f"DROP TABLE IF EXISTS {target_table}")
spark.sql(f"DROP TABLE IF EXISTS {stg_table}")
spark.sql(f"DROP TABLE IF EXISTS {target_table}_external")
spark.sql(f"DROP DATABASE IF EXISTS {database}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [16]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database}")
spark.sql(f"USE {database}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [8]:
spark.sql("""
            CREATE OR REPLACE TABLE merchants_cln (merchant_id STRING, company_name STRING, telephone STRING, created_at STRING, updated_at STRING)
            USING DELTA
            PARTITIONED BY (state STRING)
            LOCATION 's3://aws-poc-serverless-analytics/delta_lake_demo/clean/delta_dw/delta_lake_demo.db/merchants_cln/' 
            """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

### Overwrite file from raw location to clean location

In [11]:
# Set up relevant path.
inputPath = "s3://aws-poc-serverless-analytics/delta_lake_demo/test_data/merchants.csv"


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
inputSchema = "merchant_id STRING, company_name STRING, telephone STRING, state STRING, created_at STRING, updated_at STRING"

rawDF = (spark.read 
  .option("header", "true")
  .schema(inputSchema)
  .csv(inputPath) 
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [150]:
## overwrite
(rawDF.write
  .mode("overwrite")
  .format("delta")
  .partitionBy("state")
  .saveAsTable(f"{target_table}") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
## overwrite -- merchants
spark.sql(f"""select * from {target_table}""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Table or view not found: merchants_cln; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [merchants_cln], [], false

Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/session.py", line 723, in sql
    return DataFrame(self._jsparkSession.sql(sqlQuery), self._wrapped)
  File "/usr/lib/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py", line 1305, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 117, in deco
    raise converted from None
pyspark.sql.utils.AnalysisException: Table or view not found: merchants_cln; line 1 pos 14;
'Project [*]
+- 'UnresolvedRelation [merchants_cln], [], false




In [152]:
spark.sql(f"""GENERATE symlink_format_manifest FOR TABLE {target_table}""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [153]:
spark.sql(f""" ALTER TABLE {target_table} SET TBLPROPERTIES(delta.compatibility.symlinkFormatManifest.enabled=true)""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [154]:
spark.sql(f""" CREATE EXTERNAL TABLE IF NOT EXISTS {target_table}_external (id STRING, company_name STRING, telephone STRING, created_at STRING, updated_at STRING)
PARTITIONED BY (state STRING)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 's3://aws-poc-serverless-analytics/delta_lake_demo/clean/delta_dw/delta_lake_demo.db/merchants_cln/_symlink_format_manifest/' 
""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [155]:
spark.sql(f"""MSCK REPAIR TABLE {target_table}_external""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

### Read in Some New Data for SCD 1

In [104]:
miniDataInputPath = "s3://aws-poc-serverless-analytics/delta_lake_demo/test_data/merchants_SCD1.csv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [105]:
inputSchema = "id INT, company_name STRING, telephone LONG, state STRING, created_at TIMESTAMP, updated_at TIMESTAMP"

newDataDF = (spark.read 
  .option("header", "true")
  .schema(inputSchema)
  .csv(miniDataInputPath) 
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [106]:
# write using Databricks Delta format
(newDataDF.write
  .mode("overwrite")
  .format("delta")
  .partitionBy("state")
  .saveAsTable(f"{stg_table}") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [107]:
spark.sql(f"""select * from {stg_table}""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------------------+----------+-----------+--------------------+--------------------+
|   id|        company_name| telephone|      state|          created_at|          updated_at|
+-----+--------------------+----------+-----------+--------------------+--------------------+
|44083|Quick Draw Outdoo...|3182596422|        abc|2019-02-04 16:57:...|2021-02-10 02:07:...|
|44082|Slow Draw Outdoor...|3182596422|provisioned|2019-02-04 16:57:...|2021-02-11 02:07:...|
+-----+--------------------+----------+-----------+--------------------+--------------------+

In [108]:
primary_key = 'id'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
spark.sql(f""" 
          MERGE INTO {target_table}
          USING {stg_table}
          on {target_table}.{primary_key} = {stg_table}.{primary_key}
          WHEN MATCHED THEN 
              UPDATE SET *
          WHEN NOT MATCHED 
              THEN INSERT *
              
            """)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [110]:
spark.sql(f"""select * from {target_table}""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+------------+-----------+--------------------+--------------------+
|    id|        company_name|   telephone|      state|          created_at|          updated_at|
+------+--------------------+------------+-----------+--------------------+--------------------+
| 44082|Slow Draw Outdoor...|  3182596422|provisioned|2019-02-04 16:57:...|2021-02-11 02:07:...|
| 44083|Quick Draw Outdoo...|  3182596422|        abc|2019-02-04 16:57:...|2021-02-10 02:07:...|
|144524|     Rachid hamzaoui|  0639756718|       free|2020-09-21 17:18:...|2020-10-22 11:15:...|
|140294|              myself|  7149259700|       free|2020-09-05 17:41:...|2020-10-06 11:15:...|
|156444|     Only The Flyest|  4049038447|       free|2020-10-27 05:26:...|2020-11-26 11:15:...|
|177863|  Bellelyse Boutique|000-000-0000|  suspended|2020-12-31 04:56:...|2021-02-19 16:53:...|
| 50742|Thenaricalicollec...|  5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|
|176321|   CLICK MY CART LTD| 

In [112]:
spark.sql(f"""MSCK REPAIR TABLE {target_table}_external""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

### Read in Some New Data for SCD 2 

In [103]:
from pyspark.sql.functions import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
SCD2_Path = "s3a://aws-poc-serverless-analytics/dev/flattened_raw/mr/SCD2/merchants_SCD2.csv"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [72]:
inputSchema = "id INT, company_name STRING, telephone LONG, state STRING, created_at TIMESTAMP, updated_at TIMESTAMP"

rawDF = (spark.read 
  .option("header", "true")
  .schema(inputSchema)
  .csv(SCD2_Path) 
)
rawDF = rawDF.withColumn('active', lit("true"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
# write using Databricks Delta format
(rawDF.write
  .mode("overwrite") 
  .format("delta")
  .partitionBy("state")
  .option("overwriteSchema","true")
  .saveAsTable(f"{stg_table}") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
spark.sql(f""" MERGE INTO {target_table}
USING (
   SELECT {stg_table}.{primary_key} as mergeKey, {stg_table}.*
  FROM {stg_table}
  
  UNION ALL
    SELECT NULL as mergeKey, {stg_table}.*
  FROM {stg_table} JOIN {target_table}
  ON {stg_table}.{primary_key} = {target_table}.{primary_key} 
  WHERE {target_table}.active = true AND {stg_table}.updated_at <> {target_table}.updated_at 
  
) staged_updates
ON {target_table}.{primary_key} = mergeKey
WHEN MATCHED AND {target_table}.active = true AND {target_table}.updated_at <> staged_updates.updated_at THEN  
  UPDATE SET active = false
WHEN NOT MATCHED THEN 
  INSERT *
            """)



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [76]:
spark.sql(f"""select * from {target_table}""").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------------------+----------+-----------+--------------------+--------------------+------+--------+
|    id|        company_name| telephone|      state|          created_at|          updated_at|active|mergeKey|
+------+--------------------+----------+-----------+--------------------+--------------------+------+--------+
| 44082|Quick Draw Outdoo...|3182596422|       free|2019-02-04 16:57:...|2021-02-10 02:07:...|  true|    null|
| 44083|Quick Draw Outdoo...|3182596422|trial_ended|2019-02-04 16:57:...|2021-03-05 02:07:...|  true|    null|
| 44083|Quick Draw Outdoo...|3182596422|  suspended|2019-02-04 16:57:...|2021-02-10 02:07:...| false|    null|
| 44084|         godaddy llc|3182596422|provisioned|2019-02-04 16:57:...|2021-02-10 02:07:...|  true|   44084|
|176321|   CLICK MY CART LTD|6479863690|  suspended|2020-12-23 06:20:...|2020-12-24 04:53:...|  true|    null|
| 50742|Thenaricalicollec...|5049394269|  suspended|2019-03-22 03:57:...|2019-05-15 00:54:...|  true|    null|
|

In [78]:
spark.sql(f"""MSCK REPAIR TABLE {target_table}_external""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [81]:
spark.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…